# multiple regression

In [ ]:
"""
- 여러개의 특성(=독립변수=컬럼=항목=변수)을 사용한 선형회귀
- 특성이 많을수록 복잡도가 높아짐
- y = a * 특성1 + b * 특성2 + ... + 절편
"""

In [2]:
import pandas as pd
file_path = "./data/03_농어의_길이_높이_두께_데이터.csv"
df = pd.read_csv(file_path)
df.head()

,length,height,width
0,8.4,2.11,1.41
1,13.7,3.53,2.00
2,15.0,3.82,2.43
3,16.2,4.59,2.63
4,17.4,4.59,2.94


In [5]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   length   56 non-null     float64
 1    height  56 non-null     float64
 2    width   56 non-null     float64
dtypes: float64(3)
memory usage: 1.4 KB


,length,height,width
count,56.000000,56.000000,56.000000
mean,27.892857,7.862143,4.745536
std,9.021668,2.878343,1.775006
min,8.400000,2.110000,1.410000
25%,21.825000,5.690000,3.520000
50%,25.300000,6.920000,4.155000
75%,36.625000,10.850000,6.450000
max,44.000000,12.800000,8.140000


In [9]:
# df --> 2차원 배열로 변환
perch_full = df.to_numpy()
perch_full

array([[ 8.4 ,  2.11,  1.41],
       [13.7 ,  3.53,  2.  ],
       [15.  ,  3.82,  2.43],
       [16.2 ,  4.59,  2.63],
       [17.4 ,  4.59,  2.94],
       [18.  ,  5.22,  3.32],
       [18.7 ,  5.2 ,  3.12],
       [19.  ,  5.64,  3.05],
       [19.6 ,  5.14,  3.04],
       [20.  ,  5.08,  2.77],
       [21.  ,  5.69,  3.56],
       [21.  ,  5.92,  3.31],
       [21.  ,  5.69,  3.67],
       [21.3 ,  6.38,  3.53],
       [22.  ,  6.11,  3.41],
       [22.  ,  5.64,  3.52],
       [22.  ,  6.11,  3.52],
       [22.  ,  5.88,  3.52],
       [22.  ,  5.52,  4.  ],
       [22.5 ,  5.86,  3.62],
       [22.5 ,  6.79,  3.62],
       [22.7 ,  5.95,  3.63],
       [23.  ,  5.22,  3.63],
       [23.5 ,  6.28,  3.72],
       [24.  ,  7.29,  3.72],
       [24.  ,  6.38,  3.82],
       [24.6 ,  6.73,  4.17],
       [25.  ,  6.44,  3.68],
       [25.6 ,  6.56,  4.24],
       [26.5 ,  7.17,  4.14],
       [27.3 ,  8.32,  5.14],
       [27.5 ,  7.17,  4.34],
       [27.5 ,  7.05,  4.34],
       [27

In [7]:
import numpy as np
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [11]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, 
                                                                      test_size=0.3, shuffle=True)
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(39, 3) (39,)
(17, 3) (17,)


# 특성 공학 = 특성 만들기

In [ ]:
"""
<특성 생성 패키지 및 라이브러리, 함수>
- 사용되는 패키지: sklearn.preprocessing
- 사용되는 클래스: PolynomialFeatures
- 사용되는 함수: fir(), transform(), fit_transform()
- 사용되는 데이터: (종속변수를 제외한 모든 데이터)train_input, test_input (target은 그대로!)

<특성 조합 패턴>
- 최초 값: 선형방정식의 y절편값을 가장 처음에 추가, 기본값 1 사용(선형방정식 y절편의 기본값=1)
- 두번째 ~ 세번째 값: 자기 자신 그대로 적용 e.g. (2, 3)
- 네번째 값: 첫번째 기존 특성 값을 제곱한 항목을 추가 e.g. (2^2)
- 다섯번째 값: 특성끼리 곱한 값 e.g. (2 * 3)
- 여섯번째 값: 각 특성을 제곱한 값 e.g. (3^2)
- 최초값인 y절편값 1은 보통 제외하고 진행함: PolynomialFeatures(include_bias=False)

**훈련데이터로 fit한 객체로 테스트데이터를 변환해야 같은 특성 조합 패턴이됨**
- 특성을 추가한다는 의미: 차원(degree)을 기준으로 생성
- 기본적으로 사용되는 차원은 2차원(2의 제곱)
- hyperparameter 변수: degree = 2 (default)
"""

In [12]:
from sklearn.preprocessing import PolynomialFeatures

# 객체 생성
poly = PolynomialFeatures()
# 임의 데이터 사용
temp_data = [[2, 3]]
# 특성 조합을 스스로 학습하여 찾기
poly.fit(temp_data)
# 스스로 찾은 특성조합으로 변환하기
poly.transform(temp_data)

array([[1., 2., 3., 4., 6., 9.]])

In [14]:
# 특성에서 절편 제외 시키기
poly = PolynomialFeatures(include_bias=False)
poly.fit([[55, 12.49, 7.6]])
poly.transform([[55, 12.49, 7.6]])

array([[  55.    ,   12.49  ,    7.6   , 3025.    ,  686.95  ,  418.    ,
         156.0001,   94.924 ,   57.76  ]])

In [23]:
# train & test input에 특성 조합하여 생성
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print(train_poly.shape, train_target.shape)
print(test_poly.shape, test_target.shape)

(39, 9) (39,)
(17, 9) (17,)


In [24]:
# 특성 9개가 만들어진 계산 식 확인하기
print(poly.get_feature_names_out())

['x0' 'x1' 'x2' 'x0^2' 'x0 x1' 'x0 x2' 'x1^2' 'x1 x2' 'x2^2']


In [25]:
# degree=2로 다중회귀 훈련하기
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target)

LinearRegression()

In [27]:
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

0.9903988984720952
0.9685625941209276


In [ ]:
# degree=5
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

In [30]:
poly.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2', 'x0^3', 'x0^2 x1', 'x0^2 x2', 'x0 x1^2', 'x0 x1 x2',
       'x0 x2^2', 'x1^3', 'x1^2 x2', 'x1 x2^2', 'x2^3', 'x0^4', 'x0^3 x1',
       'x0^3 x2', 'x0^2 x1^2', 'x0^2 x1 x2', 'x0^2 x2^2', 'x0 x1^3',
       'x0 x1^2 x2', 'x0 x1 x2^2', 'x0 x2^3', 'x1^4', 'x1^3 x2',
       'x1^2 x2^2', 'x1 x2^3', 'x2^4', 'x0^5', 'x0^4 x1', 'x0^4 x2',
       'x0^3 x1^2', 'x0^3 x1 x2', 'x0^3 x2^2', 'x0^2 x1^3',
       'x0^2 x1^2 x2', 'x0^2 x1 x2^2', 'x0^2 x2^3', 'x0 x1^4',
       'x0 x1^3 x2', 'x0 x1^2 x2^2', 'x0 x1 x2^3', 'x0 x2^4', 'x1^5',
       'x1^4 x2', 'x1^3 x2^2', 'x1^2 x2^3', 'x1 x2^4', 'x2^5'],
      dtype=object)

In [28]:
lr = LinearRegression().fit(train_poly, train_target)
print(lr.score(train_poly, train_target), lr.score(test_poly, test_target))
#과대적합 발생

0.9999999973242714 -321458.6988690704


In [ ]:
"""
# 과대적합 해소방법
- 규제: 과대적합이 일어나지 않도록 제어하는 것
- ridge, lasso 회귀 모델: 규제의 개념을 적용한 훈련모델(주로 ridge선호)

## 규제하는 방법
- 데이터 특성들을 모두 정규화(표준화)하여 처리함 = 단위 통일

## 정규화 방법
- 일반적으로 스케일을 통일 시키기 위한 표준점수 사용
- 정규화 클래스 생성: StandardScaler(), MinMaxScaler(),... 
- 훈련 시키기: fit(훈련데이터) --> 표준점수 생성
- 변환하기: transform(훈련데이터) --> 표준점수로 변환
- 기본적으로 표준점수는 StandardScaler() 사용
--> 무엇으로 정규화할지 모를때는 StandardScaler()사용

## ridge
- from sklearn.linear_model import Ridge
- 모델: Ridge
- 계수(선형 및 다항방정식에서 x값을 제외한 값들)를 제곱한 값을 기준으로 규제 적용
- 훈련/평가/테스트: fit()/score()/predict()

## lasso
- from sklearn.linear_model import Lasso
- 모델: Lasso
- 계수(선형 및 다항방정식에서 x값을 제외한 값들)를 제곱한 값을 기준으로 규제 적용
- 훈련/평가/테스트: fit()/score()/predict()
"""

# 규제 위해 데이터 정규화

In [31]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

In [33]:
print(train_scaled.shape, test_scaled.shape)

(39, 55) (17, 55)


# ridge

In [34]:
from sklearn.linear_model import Ridge
rd = Ridge()
rd.fit(train_scaled, train_target)
print(rd.score(train_scaled, train_target), rd.score(test_scaled, test_target))

0.989810521328696 0.974018856294862
